In [ ]:
# Importar librerías necesarias
import os
import numpy as np
import scipy.io
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

# Funciones para cargar la señal y extraer características
def load_signal(datos_path):
    data = scipy.io.loadmat(datos_path)
    signal = data['signal'][0]
    fs = data['Fs'][0, 0]
    return signal, fs

def signal_feature_extraction(signal, fs):
    max_theoretical_value = 230 * np.sqrt(2)
    normalized_signal = signal / max_theoretical_value
    mean_value = np.mean(normalized_signal)
    unbias_data = normalized_signal - mean_value
    unbias_data_2 = unbias_data ** 2
    unbias_data_3 = unbias_data_2 * unbias_data
    unbias_data_4 = unbias_data_3 * unbias_data
    variance = np.var(unbias_data)
    skewness = np.mean(unbias_data_3) / (variance ** 1.5)
    kurtosis = np.mean(unbias_data_4) / (variance ** 2) - 3
    thd = np.sqrt(np.sum(np.abs(np.fft.fft(normalized_signal)[2:4])) / np.abs(np.fft.fft(normalized_signal)[1]))
    rms = np.sqrt(np.mean(normalized_signal ** 2))
    crest_factor = np.max(normalized_signal) / rms
    return variance, skewness, kurtosis, thd, crest_factor

# Recorrer carpetas y obtener características
data_path = 'ruta_a_tus_datos'  # Cambia esto por el path a tu carpeta 'data'
labels = []  # Aquí asigna las etiquetas de las carpetas, por ejemplo, 0 para 'normal', 1 para 'fault'
features = []  # Lista para almacenar características
targets = []   # Lista para almacenar etiquetas

for folder in os.listdir(data_path):
    folder_path = os.path.join(data_path, folder)
    if os.path.isdir(folder_path):
        label = 0 if folder == 'normal' else 1  # Ajusta según tu organización
        for file in os.listdir(folder_path):
            if file.endswith('.mat'):
                file_path = os.path.join(folder_path, file)
                signal, fs = load_signal(file_path)
                feature_vector = signal_feature_extraction(signal, fs)
                features.append(feature_vector)
                targets.append(label)

# Convertir a numpy arrays
features = np.array(features)
targets = np.array(targets)

In [ ]:
# Normalización de características
scaler = StandardScaler()
features_normalized = scaler.fit_transform(features)

# Dividir en conjunto de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(features_normalized, targets, test_size=0.3, random_state=42)

In [ ]:
# Crear el modelo
model = Sequential([
    Dense(32, activation='relu', input_shape=(features_normalized.shape[1],)),  # Capa de entrada
    Dense(16, activation='relu'),  # Capa oculta
    Dense(1, activation='sigmoid')  # Capa de salida (problema binario)
])

# Compilar el modelo
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Entrenar el modelo
history = model.fit(X_train, y_train, epochs=50, batch_size=16, validation_split=0.2)

# Evaluar el modelo
loss, accuracy = model.evaluate(X_test, y_test)
print(f'Loss: {loss}, Accuracy: {accuracy}')

In [ ]:
import matplotlib.pyplot as plt

# Gráfico de pérdida
plt.plot(history.history['loss'], label='Train Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()

# Gráfico de precisión
plt.plot(history.history['accuracy'], label='Train Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()
plt.show()